In [11]:
import pandas as pd
import feather

from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from keras.models import load_model


In [12]:
n_groups_per_col = 9
architecture = "2_dense_512"
spread = "spread"
path = "../data/models/ind/binary/" + architecture +"/" + spread + "/"

data_train = pd.read_feather(path + "data_train.feather")

labels_low_train = pd.read_feather(path + "labels_low_train.feather")
labels_high_train = pd.read_feather(path + "labels_high_train.feather")
labels_close_train = pd.read_feather(path + "labels_close_train.feather")

In [13]:
train_data = data_train.to_numpy()
labels_low_train = labels_low_train.to_numpy()
labels_high_train = labels_high_train.to_numpy()
labels_close_train = labels_close_train.to_numpy()

In [14]:
def fit_ind_binary_model(data_train, labels_train):
    
    model = Sequential()
    model.add(Dense(units=512, activation='relu', input_shape=(data_train.shape[1],)))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dense(labels_train.shape[1], activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    history = model.fit(data_train, labels_train, validation_split=0.2, batch_size=512, epochs=3, verbose=1)
    
    return (history, model)

In [15]:
# low
(low_history, low_model) = fit_ind_binary_model(data_train, labels_low_train)
feather.write_dataframe(pd.DataFrame.from_dict(low_history.history), path + "low_history.feather", compression = "uncompressed")
low_model.save(path + "low_model.hdf5")


Train on 3566135 samples, validate on 891534 samples
Epoch 1/3
3566135/3566135 [==============================] - 30s 8us/step - loss: 0.4721 - accuracy: 0.7725 - val_loss: 0.4651 - val_accuracy: 0.7655
Epoch 2/3
3566135/3566135 [==============================] - 30s 8us/step - loss: 0.4212 - accuracy: 0.7962 - val_loss: 0.4165 - val_accuracy: 0.7986
Epoch 3/3
3566135/3566135 [==============================] - 29s 8us/step - loss: 0.4095 - accuracy: 0.8030 - val_loss: 0.3915 - val_accuracy: 0.8112


In [16]:
# high
(high_history, high_model) = fit_ind_binary_model(train_data, labels_high_train)
feather.write_dataframe(pd.DataFrame.from_dict(high_history.history), path + "high_history.feather", compression = "uncompressed")
high_model.save(path + "high_model.hdf5")

Train on 3566135 samples, validate on 891534 samples
Epoch 1/3
3557888/3566135 [============================>.] - ETA: 0s - loss: 0.4838 - accuracy: 0.7666

KeyboardInterrupt: 

In [ ]:
# close
(close_history, close_model) = fit_ind_binary_model(train_data, labels_close_train)
feather.write_dataframe(pd.DataFrame.from_dict(close_history.history), path + "history.feather", compression = "uncompressed")
close_model.save(path + "close_model.hdf5")


In [ ]:
### Predict

# low_model = load_model("../data/models/ind/binary/low_model.hdf5")
# high_model = load_model("../data/models/ind/binary/high_model.hdf5")
# close_model = load_model("../data/models/ind/binary/close_model.hdf5")

In [11]:
data_test = pd.read_feather(path + "data_test.feather").to_numpy()

low_pred_prob = low_model.predict_proba(data_test, batch_size = 512)
feather.write_dataframe(pd.DataFrame(low_pred_prob), path + "low_pred_prob.feather", compression='uncompressed')

high_pred_prob = high_model.predict_proba(data_test, batch_size = 512)
feather.write_dataframe(pd.DataFrame(high_pred_prob), path + "high_pred_prob.feather", compression='uncompressed')

close_pred_prob = close_model.predict_proba(data_test, batch_size = 512)
feather.write_dataframe(pd.DataFrame(close_pred_prob), path + "close_pred_prob.feather", compression='uncompressed')
